# tea5poon

In [1]:
import os
import numpy as np
import tensorflow as wtf
import tensorflow.contrib.rnn as rnn
import data_utils as du

/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
EPOCHS = 32

num_train, num_val, num_test = 70, 10, 20
train_data,val_data,test_data = du.create_dicts(num_train, num_val, num_test)

In [3]:
MODEL_TYPE = 'tea5poon' # choices: model (basic non-diff), diff, lstm, lstm-p, three
OVERFIT = False

MODEL_NAME = MODEL_TYPE + '-000'

BATCH_SIZE = 1

In [4]:
def encoder_fn(inputs):
    conv1_1 = wtf.layers.conv2d(inputs,
                                filters=64,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv1_1')
    conv1_2 = wtf.layers.conv2d(conv1_1,
                                filters=64,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv1_2')
    pool1 = wtf.layers.max_pooling2d(conv1_2,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool1')
        
    conv2_1 = wtf.layers.conv2d(pool1,
                                filters=128,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv2_1')
    conv2_2 = wtf.layers.conv2d(conv2_1,
                                filters=128,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv2_2')
    pool2 = wtf.layers.max_pooling2d(conv2_2,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool2')
        
    conv3_1 = wtf.layers.conv2d(pool2,
                                filters=256,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv3_1')
    conv3_2 = wtf.layers.conv2d(conv3_1,
                                filters=256,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv3_2')
    conv3_3 = wtf.layers.conv2d(conv3_2,
                                filters=256,
                                kernel_size=3,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv3_3')
    pool3 = wtf.layers.max_pooling2d(conv3_3,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool3')
        
    conv4_1 = wtf.layers.conv2d(pool3,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv4_1')
    conv4_2 = wtf.layers.conv2d(conv4_1,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv4_2')
    conv4_3 = wtf.layers.conv2d(conv4_2,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv4_3')
    pool4 = wtf.layers.max_pooling2d(conv4_3,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool4')
    
    conv5_1 = wtf.layers.conv2d(pool4,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv5_1')
    conv5_2 = wtf.layers.conv2d(conv5_1,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv5_2')
    conv5_3 = wtf.layers.conv2d(conv5_2,
                                filters=512,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv5_3')
    pool5 = wtf.layers.max_pooling2d(conv5_3,
                                     pool_size=2,
                                     strides=2,
                                     padding='same',
                                     name='pool5')
    
    conv6_1 = wtf.layers.conv2d(pool5,
                                filters=1,
                                kernel_size=1,
                                strides=1,
                                padding='same',
                                activation=wtf.nn.relu,
                                kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                name='conv6_1')
        
    dout = wtf.layers.dropout(conv6_1,
                              rate=0.0)
    
    outputs = dout
    return outputs, pool1, pool3, pool4

In [5]:
def decoder_fn(inputs, pool1, pool3, pool4):
    deconv1 = wtf.layers.conv2d_transpose(inputs,
                                          filters=pool4.get_shape().as_list()[-1],
                                          kernel_size=4,
                                          strides=2,
                                          padding='same')
    deconv1_skip = wtf.add(deconv1, pool4)
    
    deconv2 = wtf.layers.conv2d_transpose(deconv1_skip,
                                          filters=pool3.get_shape().as_list()[-1],
                                          kernel_size=4,
                                          strides=2,
                                          padding='same')
    deconv2_skip = wtf.add(deconv2, pool3)
    
    deconv3 = wtf.layers.conv2d_transpose(deconv2_skip,
                                          filters=1,
                                          kernel_size=16,
                                          strides=8,
                                          padding='same')
#     deconv3_skip = wtf.add(deconv3, pool1)
    
    outputs = deconv3
    return outputs

In [6]:
# t0,t1,t2 -> t3 + LSTM

def model_fn(features,
             labels,
             mode,
             params):
    with wtf.device('/device:GPU:1'):
        inputs = features
        print(inputs)
        inputs_tr = wtf.transpose(inputs, [2, 3, 4, 1, 0])
        inputs_re = wtf.reshape(inputs_tr, [192, 224, 3, -1])
        inputs_NHWC = wtf.transpose(inputs_re, [3, 0, 1, 2])
        print(inputs_NHWC)
        dout, pool1, pool3, pool4 = encoder_fn(inputs_NHWC) # dout (15*N, 6, 7, 1)
        dout_tr = wtf.transpose(dout, [1, 2, 3, 0])
        dout_re = wtf.reshape(dout_tr, [6*7*1, 15, -1])
        dout_NTC = wtf.transpose(dout_re, [2, 1, 0])
        lstm_cell = rnn.LSTMCell(num_units=6*7*1,
                                 forget_bias=1.0,
                                 reuse=None,
                                 name=None)
        rnn_outputs, _ = wtf.nn.dynamic_rnn(cell=lstm_cell,
                                            inputs=dout_NTC,
                                            dtype=wtf.float32)
#         print(rnn_outputs) # I'm not sure what shape the rnn outputs
        rnn_tr = wtf.transpose(rnn_outputs, [2, 1, 0])
        rnn_re = wtf.reshape(rnn_tr, [6, 7, 1, -1])
        rnn_NHWC = wtf.transpose(rnn_re, [3, 0, 1, 2])
        print(rnn_NHWC)
        deconv3 = decoder_fn(rnn_NHWC, pool1, pool3, pool4)
#         outputs = wtf.add(deconv2, inputs)
        dc3_tr = wtf.transpose(deconv3, [1, 2, 3, 0])
        dc3_re = wtf.reshape(dc3_tr, [192, 224, 1, 15, -1])
        outputs = wtf.transpose(dc3_re, [4, 3, 0, 1, 2])
#         outputs = inputs + outputs # Eh I'm not sure this does anything
        print(outputs)

#         wtf.summary.image('input', inputs, max_outputs=200)
#         wtf.summary.image('outputs', outputs, max_outputs=200)
#         wtf.summary.image('labels', labels, max_outputs=200)
        
        # Adds concatenated inputs and outputs to be displayed in TensorBoard
        disp00 = wtf.concat([wtf.expand_dims(inputs[:, 0,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 0,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 0,:,:,2], -1),
                             labels[:, 0], outputs[:, 0]], 1)
        disp01 = wtf.concat([wtf.expand_dims(inputs[:, 1,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 1,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 1,:,:,2], -1),
                             labels[:, 1], outputs[:, 1]], 1)
        disp02 = wtf.concat([wtf.expand_dims(inputs[:, 2,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 2,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 2,:,:,2], -1),
                             labels[:, 2], outputs[:, 2]], 1)
        disp03 = wtf.concat([wtf.expand_dims(inputs[:, 3,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 3,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 3,:,:,2], -1),
                             labels[:, 3], outputs[:, 3]], 1)
        disp04 = wtf.concat([wtf.expand_dims(inputs[:, 4,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 4,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 4,:,:,2], -1),
                             labels[:, 4], outputs[:, 4]], 1)
        disp05 = wtf.concat([wtf.expand_dims(inputs[:, 5,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 5,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 5,:,:,2], -1),
                             labels[:, 5], outputs[:, 5]], 1)
        disp06 = wtf.concat([wtf.expand_dims(inputs[:, 6,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 6,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 6,:,:,2], -1),
                             labels[:, 6], outputs[:, 6]], 1)
        disp07 = wtf.concat([wtf.expand_dims(inputs[:, 7,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 7,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 7,:,:,2], -1),
                             labels[:, 7], outputs[:, 7]], 1)
        disp08 = wtf.concat([wtf.expand_dims(inputs[:, 8,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 8,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 8,:,:,2], -1),
                             labels[:, 8], outputs[:, 8]], 1)
        disp09 = wtf.concat([wtf.expand_dims(inputs[:, 9,:,:,0], -1),
                             wtf.expand_dims(inputs[:, 9,:,:,1], -1),
                             wtf.expand_dims(inputs[:, 9,:,:,2], -1),
                             labels[:, 9], outputs[:, 9]], 1)
        disp10 = wtf.concat([wtf.expand_dims(inputs[:,10,:,:,0], -1),
                             wtf.expand_dims(inputs[:,10,:,:,1], -1),
                             wtf.expand_dims(inputs[:,10,:,:,2], -1),
                             labels[:,10], outputs[:,10]], 1)
        disp11 = wtf.concat([wtf.expand_dims(inputs[:,11,:,:,0], -1),
                             wtf.expand_dims(inputs[:,11,:,:,1], -1),
                             wtf.expand_dims(inputs[:,11,:,:,2], -1),
                             labels[:,11], outputs[:,11]], 1)
        disp12 = wtf.concat([wtf.expand_dims(inputs[:,12,:,:,0], -1),
                             wtf.expand_dims(inputs[:,12,:,:,1], -1),
                             wtf.expand_dims(inputs[:,12,:,:,2], -1),
                             labels[:,12], outputs[:,12]], 1)
        disp13 = wtf.concat([wtf.expand_dims(inputs[:,13,:,:,0], -1),
                             wtf.expand_dims(inputs[:,13,:,:,1], -1),
                             wtf.expand_dims(inputs[:,13,:,:,2], -1),
                             labels[:,13], outputs[:,13]], 1)
        disp14 = wtf.concat([wtf.expand_dims(inputs[:,14,:,:,0], -1),
                             wtf.expand_dims(inputs[:,14,:,:,1], -1),
                             wtf.expand_dims(inputs[:,14,:,:,2], -1),
                             labels[:,14], outputs[:,14]], 1)
        disp = wtf.concat([disp00, disp01, disp02, disp03, disp04, disp05, disp06,
                           disp07, disp08, disp09, disp10, disp11, disp12, disp13,
                           disp14], 2)
        
        wtf.summary.image('concat', disp, max_outputs=200)
    
        predictions = {'results': outputs}
    
        if mode == wtf.estimator.ModeKeys.PREDICT:
            export_outputs = {
                'predictions': wtf.estimator.export.PredictOutput(outputs)
            }
            return wtf.estimator.EstimatorSpec(
                mode=mode,
                predictions=outputs
            )
    
        # loss = wtf.losses.mean_squared_error(labels, outputs)
#         loss = wtf.reduce_sum(wtf.abs(labels - outputs), axis=[1,2,3])
        loss = wtf.reduce_sum(wtf.abs(labels - outputs), axis=[2,3,4])
        loss = wtf.reduce_mean(loss)
        
        global_step = wtf.train.get_global_step()
        lr = wtf.train.exponential_decay(params.learning_rate, global_step, 10000, 0.96, staircase=True)
        
        wtf.summary.scalar('learning_rate', lr)
#         wtf.summary.scalar('learning_rate', params.learning_rate)
        
        optimizer = wtf.train.AdamOptimizer(
#             learning_rate=params.learning_rate,
            learning_rate=lr,
            beta1=params.beta1,
            beta2=params.beta2
        )
    
        update_ops = wtf.get_collection(wtf.GraphKeys.UPDATE_OPS)
        with wtf.control_dependencies(update_ops):
            train_op = optimizer.minimize(
                loss=loss, global_step=wtf.train.get_global_step())
    
        eval_metric_ops = {
            "mse_i1": wtf.metrics.mean_squared_error(labels, inputs[:,:,:,:,0]),
            "mse_i2": wtf.metrics.mean_squared_error(labels, inputs[:,:,:,:,1]),
            "mse_i3": wtf.metrics.mean_squared_error(labels, inputs[:,:,:,:,2]),
            "mse_o": wtf.metrics.mean_squared_error(labels, outputs)
        }
        
        eval_summary_hook = wtf.train.SummarySaverHook(
                                save_steps=1,
                                output_dir= "summary/train/" + MODEL_NAME + "/eval",
                                scaffold=wtf.train.Scaffold(summary_op=wtf.summary.merge_all()))
        
        evaluation_hooks=[]
        evaluation_hooks.append(eval_summary_hook)
    
        estimator_spec = wtf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=eval_metric_ops,
            evaluation_hooks=evaluation_hooks
        )
        
        return estimator_spec

In [7]:
def main(_):
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    
    wtf.logging.set_verbosity(wtf.logging.INFO)

    hparams = wtf.contrib.training.HParams(
        #num_epochs=1,
        #batch_size = 80,
        #forget_bias=1.0,
        learning_rate=0.0001,
        beta1=0.9,
        beta2=0.999
    )
    
    session_config = wtf.ConfigProto()
    session_config.gpu_options.allow_growth = True
    session_config.allow_soft_placement = True
    
    run_config = wtf.estimator.RunConfig(
        log_step_count_steps=10,
        save_summary_steps=10,
        tf_random_seed=19830610,
        model_dir=os.path.join('summary', 'train', MODEL_NAME),
        session_config=session_config
    )
    
    estimator = wtf.estimator.Estimator(
        model_fn=model_fn, params=hparams, config=run_config)
    
    train_dataset = du.create_dataset_tea5poon('train', train_data)
    train_dataset = train_dataset.shuffle(500)
    train_dataset = train_dataset.batch(BATCH_SIZE)
    train_dataset = train_dataset.repeat(EPOCHS)
    
    val_dataset = du.create_dataset_tea5poon('val', val_data)
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.repeat(1)
    
    test_dataset = du.create_dataset_tea5poon('test', test_data)
    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.repeat(1)
    
    def train_input_fn():
        train_iterator = train_dataset.make_one_shot_iterator()
        features, labels = train_iterator.get_next()
        return features, labels
    
    def val_input_fn():
        val_iterator = val_dataset.make_one_shot_iterator()
        features, labels = val_iterator.get_next()
        return features, labels
    
    def test_input_fn():
        test_iterator = test_dataset.make_one_shot_iterator()
        features, labels = test_iterator.get_next()
        return features, labels
    
    estimator.train(input_fn=train_input_fn, max_steps=None)
#     estimator.evaluate(input_fn=val_input_fn)
#     estimator.evaluate(input_fn=test_input_fn)

In [8]:
if __name__ == '__main__':
    wtf.app.run()

INFO:tensorflow:Using config: {'_model_dir': 'summary/train/tea5poon-000', '_tf_random_seed': 19830610, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9e16e91780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Tensor("IteratorGetNext:0", shape=(?, 15, 192, 224, 3), dtype=float32, device=/device:CPU:0)
Tensor("transpose_1:0", shape=(?, 192, 224, 3), dtype=float32, device=/device:GPU:1)
Tensor("transpose_5:0", shape=(?, 6, 7, 1), dtype=float32, device=/device:GPU:1)
Tensor("transpose_7:0", shape=(?, 15, 19

INFO:tensorflow:global_step/sec: 3.98376
INFO:tensorflow:global_step/sec: 3.94164
INFO:tensorflow:global_step/sec: 3.46618
INFO:tensorflow:loss = 515.2386, step = 1500 (26.270 sec)
INFO:tensorflow:global_step/sec: 3.50114
INFO:tensorflow:global_step/sec: 3.49977
INFO:tensorflow:global_step/sec: 3.58745
INFO:tensorflow:global_step/sec: 3.7827
INFO:tensorflow:global_step/sec: 3.69386
INFO:tensorflow:global_step/sec: 3.51205
INFO:tensorflow:global_step/sec: 3.67369
INFO:tensorflow:global_step/sec: 4.18093
INFO:tensorflow:global_step/sec: 3.91179
INFO:tensorflow:global_step/sec: 3.40365
INFO:tensorflow:loss = 647.9733, step = 1600 (27.308 sec)
INFO:tensorflow:global_step/sec: 3.64854
INFO:tensorflow:global_step/sec: 4.01441
INFO:tensorflow:global_step/sec: 3.65221
INFO:tensorflow:global_step/sec: 4.01845
INFO:tensorflow:global_step/sec: 3.82771
INFO:tensorflow:global_step/sec: 3.8982
INFO:tensorflow:global_step/sec: 4.19504
INFO:tensorflow:global_step/sec: 3.91492
INFO:tensorflow:global_st

INFO:tensorflow:global_step/sec: 3.71924
INFO:tensorflow:global_step/sec: 3.62616
INFO:tensorflow:global_step/sec: 3.75615
INFO:tensorflow:global_step/sec: 3.75132
INFO:tensorflow:global_step/sec: 4.30487
INFO:tensorflow:global_step/sec: 3.68933
INFO:tensorflow:global_step/sec: 4.20081
INFO:tensorflow:global_step/sec: 3.87764
INFO:tensorflow:global_step/sec: 4.282
INFO:tensorflow:global_step/sec: 4.12616
INFO:tensorflow:loss = 984.74164, step = 3300 (25.528 sec)
INFO:tensorflow:global_step/sec: 3.77947
INFO:tensorflow:global_step/sec: 3.74746
INFO:tensorflow:global_step/sec: 3.71338
INFO:tensorflow:global_step/sec: 4.19719
INFO:tensorflow:global_step/sec: 4.25559
INFO:tensorflow:global_step/sec: 3.65364
INFO:tensorflow:global_step/sec: 3.91391
INFO:tensorflow:global_step/sec: 3.52035
INFO:tensorflow:global_step/sec: 3.7569
INFO:tensorflow:global_step/sec: 3.76007
INFO:tensorflow:loss = 624.39154, step = 3400 (26.192 sec)
INFO:tensorflow:global_step/sec: 3.58573
INFO:tensorflow:global_s

INFO:tensorflow:global_step/sec: 3.9507
INFO:tensorflow:global_step/sec: 3.71833
INFO:tensorflow:global_step/sec: 3.4281
INFO:tensorflow:global_step/sec: 3.87971
INFO:tensorflow:global_step/sec: 3.68485
INFO:tensorflow:global_step/sec: 3.91629
INFO:tensorflow:loss = 794.63214, step = 5000 (26.895 sec)
INFO:tensorflow:global_step/sec: 3.98024
INFO:tensorflow:global_step/sec: 4.08911
INFO:tensorflow:global_step/sec: 4.25149
INFO:tensorflow:global_step/sec: 4.07959
INFO:tensorflow:global_step/sec: 4.14373
INFO:tensorflow:global_step/sec: 3.82417
INFO:tensorflow:global_step/sec: 3.76095
INFO:tensorflow:global_step/sec: 3.87921
INFO:tensorflow:global_step/sec: 4.15877
INFO:tensorflow:global_step/sec: 3.81318
INFO:tensorflow:loss = 616.3194, step = 5100 (25.054 sec)
INFO:tensorflow:global_step/sec: 6.64911
INFO:tensorflow:global_step/sec: 6.87092
INFO:tensorflow:global_step/sec: 6.79616
INFO:tensorflow:global_step/sec: 6.94668
INFO:tensorflow:global_step/sec: 6.60509
INFO:tensorflow:global_s

INFO:tensorflow:loss = 637.496, step = 6700 (14.654 sec)
INFO:tensorflow:global_step/sec: 6.41327
INFO:tensorflow:global_step/sec: 6.63021
INFO:tensorflow:global_step/sec: 6.93858
INFO:tensorflow:global_step/sec: 6.58629
INFO:tensorflow:global_step/sec: 6.9532
INFO:tensorflow:global_step/sec: 6.96284
INFO:tensorflow:global_step/sec: 7.23807
INFO:tensorflow:global_step/sec: 6.69875
INFO:tensorflow:global_step/sec: 6.47515
INFO:tensorflow:global_step/sec: 6.39001
INFO:tensorflow:loss = 575.63666, step = 6800 (14.884 sec)
INFO:tensorflow:global_step/sec: 6.55942
INFO:tensorflow:global_step/sec: 6.45087
INFO:tensorflow:global_step/sec: 6.79642
INFO:tensorflow:global_step/sec: 6.46589
INFO:tensorflow:global_step/sec: 7.05854
INFO:tensorflow:global_step/sec: 6.05034
INFO:tensorflow:global_step/sec: 6.1114
INFO:tensorflow:global_step/sec: 6.80686
INFO:tensorflow:global_step/sec: 6.85584
INFO:tensorflow:global_step/sec: 7.24582
INFO:tensorflow:loss = 1226.4574, step = 6900 (15.107 sec)
INFO:te

...

INFO:tensorflow:global_step/sec: 4.48896
INFO:tensorflow:global_step/sec: 4.64742
INFO:tensorflow:loss = 359.3996, step = 22300 (21.353 sec)
INFO:tensorflow:global_step/sec: 4.89962
INFO:tensorflow:global_step/sec: 4.70126
INFO:tensorflow:global_step/sec: 4.84758
INFO:tensorflow:global_step/sec: 4.99533
INFO:tensorflow:global_step/sec: 4.4756
INFO:tensorflow:global_step/sec: 5.10573
INFO:tensorflow:global_step/sec: 4.71413
INFO:tensorflow:global_step/sec: 4.64304
INFO:tensorflow:global_step/sec: 4.69618
INFO:tensorflow:Saving checkpoints for 22401 into summary/train/tea5poon-000/model.ckpt.
INFO:tensorflow:global_step/sec: 0.0598425
INFO:tensorflow:loss = 435.4806, step = 22400 (185.935 sec)
INFO:tensorflow:global_step/sec: 3.39938
INFO:tensorflow:global_step/sec: 3.19892
INFO:tensorflow:global_step/sec: 3.07882
INFO:tensorflow:global_step/sec: 3.13993
INFO:tensorflow:global_step/sec: 3.29749
INFO:tensorflow:global_step/sec: 2.78381
INFO:tensorflow:global_step/sec: 3.35489
INFO:tensorf

INFO:tensorflow:global_step/sec: 3.31347
INFO:tensorflow:global_step/sec: 2.927
INFO:tensorflow:global_step/sec: 3.29411
INFO:tensorflow:global_step/sec: 3.05696
INFO:tensorflow:global_step/sec: 3.03773
INFO:tensorflow:global_step/sec: 3.17684
INFO:tensorflow:global_step/sec: 3.09959
INFO:tensorflow:global_step/sec: 3.09687
INFO:tensorflow:global_step/sec: 3.12364
INFO:tensorflow:global_step/sec: 2.77892
INFO:tensorflow:loss = 611.83307, step = 24100 (32.435 sec)
INFO:tensorflow:global_step/sec: 3.13506
INFO:tensorflow:global_step/sec: 3.02138
INFO:tensorflow:global_step/sec: 3.11336
INFO:tensorflow:global_step/sec: 3.04413
INFO:tensorflow:global_step/sec: 3.14626
INFO:tensorflow:global_step/sec: 3.11376
INFO:tensorflow:global_step/sec: 3.1435
INFO:tensorflow:global_step/sec: 3.05726
INFO:tensorflow:global_step/sec: 3.20105
INFO:tensorflow:global_step/sec: 3.14763
INFO:tensorflow:loss = 521.9912, step = 24200 (32.139 sec)
INFO:tensorflow:global_step/sec: 2.72188
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 3.08525
INFO:tensorflow:global_step/sec: 3.1428
INFO:tensorflow:global_step/sec: 3.12818
INFO:tensorflow:global_step/sec: 2.98146
INFO:tensorflow:global_step/sec: 3.08708
INFO:tensorflow:global_step/sec: 3.16233
INFO:tensorflow:loss = 468.915, step = 25800 (32.564 sec)
INFO:tensorflow:global_step/sec: 3.25693
INFO:tensorflow:global_step/sec: 3.01331
INFO:tensorflow:global_step/sec: 3.07533
INFO:tensorflow:global_step/sec: 2.72946
INFO:tensorflow:global_step/sec: 3.16874
INFO:tensorflow:global_step/sec: 3.06731
INFO:tensorflow:global_step/sec: 3.13066
INFO:tensorflow:global_step/sec: 3.1225
INFO:tensorflow:global_step/sec: 3.06427
INFO:tensorflow:global_step/sec: 3.17655
INFO:tensorflow:loss = 584.73956, step = 25900 (32.529 sec)
INFO:tensorflow:global_step/sec: 3.06909
INFO:tensorflow:global_step/sec: 3.11889
INFO:tensorflow:global_step/sec: 3.01044
INFO:tensorflow:global_step/sec: 3.28964
INFO:tensorflow:global_step/sec: 3.17596
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 3.1092
INFO:tensorflow:global_step/sec: 3.11776
INFO:tensorflow:loss = 377.13022, step = 27500 (32.093 sec)
INFO:tensorflow:global_step/sec: 7.50194
INFO:tensorflow:global_step/sec: 6.78979
INFO:tensorflow:global_step/sec: 6.35893
INFO:tensorflow:global_step/sec: 6.86057
INFO:tensorflow:global_step/sec: 7.0974
INFO:tensorflow:global_step/sec: 6.53128
INFO:tensorflow:global_step/sec: 6.4301
INFO:tensorflow:global_step/sec: 6.60231
INFO:tensorflow:global_step/sec: 6.63378
INFO:tensorflow:global_step/sec: 6.76816
INFO:tensorflow:loss = 325.94345, step = 27600 (14.831 sec)
INFO:tensorflow:global_step/sec: 6.82985
INFO:tensorflow:global_step/sec: 6.89551
INFO:tensorflow:global_step/sec: 6.8548
INFO:tensorflow:global_step/sec: 6.27757
INFO:tensorflow:global_step/sec: 7.02998
INFO:tensorflow:global_step/sec: 6.09889
INFO:tensorflow:global_step/sec: 6.65321
INFO:tensorflow:global_step/sec: 6.5372
INFO:tensorflow:global_step/sec: 6.4403
INFO:tensorflow:global_st

ResourceExhaustedError: summary/train/tea5poon-000/model.ckpt-28001_temp_cc9a34a5a54c48df83cd0ac1aff6e653; No space left on device
	 [[Node: save/SaveV2 = SaveV2[dtypes=[DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](save/ShardedFilename, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, global_step/_765)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'save/SaveV2', defined at:
  File "/home/noodles/.conda/envs/turntable/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/noodles/.conda/envs/turntable/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/trai